<a href="https://colab.research.google.com/github/leeje008/ML_final_project/blob/main/Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import  confusion_matrix,recall_score,precision_score,accuracy_score
from imblearn.metrics import geometric_mean_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier  
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE




def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc = roc_auc_score(y_test, pred)
    g_mean = geometric_mean_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:4f}, 정밀도 {1:4f}, 재현율 {2:4f}, AUC {3:4f},G_MEAN {4:4f}'.format(accuracy , precision , recall,roc,g_mean))
    

def get_model_train_eval(model,ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)



import warnings
warnings.filterwarnings(action='ignore')

In [14]:
df = pd.read_csv('/content/drive/MyDrive/Loan_Train.csv')
df.head()

df = df.drop('Id',axis = 1)


from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

data = df


for e in data.columns:
    if data[e].dtype == 'object':
        labelEncoder.fit(list(data[e].values))
        data[e] = labelEncoder.transform(data[e].values)
        
        # Accommodate the data that has been changed
        df = data
        
y = df.Risk_Flag
X = df.drop('Risk_Flag', axis=1)


X_train, X_test, y_tain, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   random_state=101)



# Hyperpameter tunning

In [46]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, make_scorer
start2 = time.time()

c = X_train.columns

start1 = time.time()


pipeline = Pipeline(steps= [("SMOTE", SMOTE()),
                            ("RandomForestClassifier", RandomForestClassifier())
                            ])

param_grid = {
    "SMOTE__sampling_strategy": [0.5,  1],
    "RandomForestClassifier__n_estimators": [100,500,1000]
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 3)
gs_pipeline.fit(X_train, y_train)

end1 = time.time()

print(f"{end1 - start1:.5f} sec")

# Store the best model




best_model = gs_pipeline.best_estimator_

end2 = time.time()

print(f"{end2 - start2:.5f} sec")


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END RandomForestClassifier__n_estimators=100, SMOTE__sampling_strategy=0.5; total time=  20.2s
[CV] END RandomForestClassifier__n_estimators=100, SMOTE__sampling_strategy=0.5; total time=  20.5s
[CV] END RandomForestClassifier__n_estimators=100, SMOTE__sampling_strategy=0.5; total time=  20.7s
[CV] END RandomForestClassifier__n_estimators=100, SMOTE__sampling_strategy=1; total time=  28.1s
[CV] END RandomForestClassifier__n_estimators=100, SMOTE__sampling_strategy=1; total time=  28.0s
[CV] END RandomForestClassifier__n_estimators=100, SMOTE__sampling_strategy=1; total time=  27.3s
[CV] END RandomForestClassifier__n_estimators=500, SMOTE__sampling_strategy=0.5; total time= 1.7min
[CV] END RandomForestClassifier__n_estimators=500, SMOTE__sampling_strategy=0.5; total time= 1.7min
[CV] END RandomForestClassifier__n_estimators=500, SMOTE__sampling_strategy=0.5; total time= 1.7min
[CV] END RandomForestClassifier__n_estimators=

In [47]:
y_validation_preds = best_model.predict(X_test)
roc_auc_score(y_val, y_validation_preds)

0.8552323112618312